In [5]:
print(1)

1


In [6]:
# This will run the files:

import subprocess

# gateway_proc = subprocess.Popen(['python3', 'gateway.py'])
# orderbook_proc = subprocess.Popen(['python3', 'orderbook.py'])

# gateway_proc.wait()
# orderbook_proc.wait()

In [7]:
import fcntl
import numpy as np
from multiprocessing import shared_memory

In [11]:
from shared_memory_utils import SharedPriceBook

# Create instance
price_book = SharedPriceBook(symbols=[])

# Read a specific symbol
aapl = price_book.read("AAPL")
print(f"AAPL price: ${aapl}")

# Read multiple symbols at once
symbols_to_check = ["AAPL", "MSFT", "GOOG", "AMZN"]
prices = {sym: price_book.read(sym) for sym in symbols_to_check}

for symbol, price in prices.items():
    print(f"{symbol}: ${price:.2f}")

AAPL price: $200.68
AAPL: $200.68
MSFT: $535.08
GOOG: $367.65
AMZN: $359.60


In [19]:
# price_book.update("AAPL", 500)

aapl = price_book.read("AAPL")
print(f"AAPL price: ${aapl}")

AAPL price: $501.37


# Timing

In [4]:
import time
from shared_memory_utils import SharedPriceBook
import socket, json

price_book = SharedPriceBook(symbols=[])
tick_times, order_times = [], []

# Monitor prices
last_price = price_book.read("AAPL")
start = time.time()
while time.time() - start < 10:
    price = price_book.read("AAPL")
    if price != last_price:
        tick_times.append(time.time())
        last_price = price
    time.sleep(0.01)

# Monitor orders
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(("127.0.0.1", 10000))
sock.settimeout(1)
buffer = b''
start = time.time()
while time.time() - start < 20:
    try:
        data = sock.recv(4096)
        if data:
            buffer += data
            while b'*' in buffer:
                msg, buffer = buffer.split(b'*', 1)
                if msg and b'"symbol":"AAPL"' in msg:
                    order_times.append(time.time())
    except: pass

# Calculate latency
latencies = [(o - t) * 1000 for t, o in zip(tick_times[-len(order_times):], order_times) if o > t]
print(f"Avg latency: {sum(latencies)/len(latencies):.2f} ms" if latencies else "No data")

No data


In [8]:
import time
from shared_memory_utils import SharedPriceBook

price_book = SharedPriceBook(symbols=[])
last_price = price_book.read("AAPL")
ticks = 0
start = time.time()

while time.time() - start < 10:
    price = price_book.read("AAPL")
    if price != last_price:
        ticks += 1
        last_price = price
    time.sleep(0.001)

print(f"Throughput: {ticks / 10:.1f} ticks/sec")

Throughput: 1.0 ticks/sec


In [6]:
import json
from multiprocessing import shared_memory

with open("market_meta.json") as f:
    meta = json.load(f)

shm = shared_memory.SharedMemory(name=meta['shm_name'], create=False)
print(f"Memory footprint: {shm.size} bytes ({shm.size/1024:.2f} KB)")
shm.close()

Memory footprint: 4096 bytes (4.00 KB)


In [7]:
from shared_memory_utils import SharedPriceBook
import time

price_book = SharedPriceBook(symbols=[])

# Test missing symbol
missing = price_book.read("INVALID")
print(f"Missing symbol: {missing}")

# Test during connection issues (read should still work)
for _ in range(5):
    price = price_book.read("AAPL")
    print(f"Price during disruption: {price}")
    time.sleep(0.5)

Missing symbol: None
Price during disruption: 204.95
Price during disruption: 204.95
Price during disruption: 205.0
Price during disruption: 205.0
Price during disruption: 204.7
